<a href="https://colab.research.google.com/github/root-git/stratascratch-sql-challenges/blob/main/4_User_Streaks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## User Streaks

Provided a table with user id and the dates they visited the platform, find the top 3 users with the longest continuous streak of visiting the platform as of August 10, 2022. Output the user ID and the length of the streak.

In case of a tie, display all users with the top three longest streaks.

**Original Question Link:**  
[StrataScratch ID 2131 – First Day Retention Rate](https://platform.stratascratch.com/coding/2131-user-streaks?code_type=1)

---

# Table Schema

#### `user_streaks`

| Column          | Type    | Description                               |
|-----------------|---------|-------------------------------------------|
| `date_visited`  | `date`  | Date the user visited the platform        |
| `user_id`       | `text`  | ID of the user                            |

---

# Thought Process

1. Remove duplicate visits.
2. Filter records to `2022-08-10`.
3. For each `user_id`, sort their visits by date.
4. Detect streak breaks.
5. Count the length of each streak.
6. Get the longest streak per user.
7. Rank users by their longest streaks.


In [1]:
import pandas as pd

#create mock data with edge cases
data = {
    "user_id": [
        "A", "A", "A", "A", "A",
        "B", "B", "B", "B", "B",
        "C", "C", "C", "C",
        "D", "D", "D", "D", "D", "D",
        "E", "E", "E",
        "F",
        "G", "G", "G", "G", "G"
    ],
    "date_visited": [
        "2022-08-06", "2022-08-07", "2022-08-08", "2022-08-09", "2022-08-10",
        "2022-08-03", "2022-08-04", "2022-08-04", "2022-08-05", "2022-08-06",
        "2022-08-01", "2022-08-02", "2022-08-04", "2022-08-05",
        "2022-08-05", "2022-08-06", "2022-08-07", "2022-08-08", "2022-08-09", "2022-08-10",
        "2022-08-01", "2022-08-03", "2022-08-05",
        "2022-08-04",
        "2022-08-11", "2022-08-12", "2022-08-13", "2022-08-14", "2022-08-15"
    ]
}

#Create DataFrame
df = pd.DataFrame(data)

df["date_visited"] = pd.to_datetime(df["date_visited"])

In [2]:
import sqlite3

# Load into SQLite (in-memory)
conn = sqlite3.connect(":memory:")
df.to_sql("user_streaks", conn, index=False, if_exists="replace")

# Show preview
print(pd.read_sql("SELECT * FROM user_streaks", conn))


   user_id         date_visited
0        A  2022-08-06 00:00:00
1        A  2022-08-07 00:00:00
2        A  2022-08-08 00:00:00
3        A  2022-08-09 00:00:00
4        A  2022-08-10 00:00:00
5        B  2022-08-03 00:00:00
6        B  2022-08-04 00:00:00
7        B  2022-08-04 00:00:00
8        B  2022-08-05 00:00:00
9        B  2022-08-06 00:00:00
10       C  2022-08-01 00:00:00
11       C  2022-08-02 00:00:00
12       C  2022-08-04 00:00:00
13       C  2022-08-05 00:00:00
14       D  2022-08-05 00:00:00
15       D  2022-08-06 00:00:00
16       D  2022-08-07 00:00:00
17       D  2022-08-08 00:00:00
18       D  2022-08-09 00:00:00
19       D  2022-08-10 00:00:00
20       E  2022-08-01 00:00:00
21       E  2022-08-03 00:00:00
22       E  2022-08-05 00:00:00
23       F  2022-08-04 00:00:00
24       G  2022-08-11 00:00:00
25       G  2022-08-12 00:00:00
26       G  2022-08-13 00:00:00
27       G  2022-08-14 00:00:00
28       G  2022-08-15 00:00:00


In [8]:
# Replace with your SQL query below
query = """ SELECT * FROM user_streaks"""

result_df = pd.read_sql(query, conn)

In [99]:
query = """
WITH unique_visits AS
(
    SELECT
      user_id,
      date_visited
    FROM   user_streaks
    WHERE  date_visited <=  '2022-08-10'
    GROUP BY user_id, date_visited
),
detected_streaks AS
(
  SELECT
    user_id,
    date_visited,
    CASE
      WHEN JULIANDAY(date_visited) - JULIANDAY(LAG(date_visited) OVER (PARTITION BY user_id ORDER BY date_visited)) = 1 THEN 0
      ELSE 1
      END AS streak_marker
  FROM unique_visits
),
streak_groups AS
(
  SELECT
    user_id,
    date_visited,
  SUM(streak_marker) OVER (PARTITION BY user_id ORDER BY date_visited) AS streak_id
  FROM detected_streaks
),
streak_lengths AS
(
SELECT
  user_id,
  streak_id,
  COUNT(*) AS streak_length,
  DENSE_RANK() OVER  (ORDER BY COUNT(*) DESC) AS streak_rank
FROM streak_groups
GROUP BY user_id, streak_id
)
SELECT
  user_id,
  streak_length
FROM streak_lengths
WHERE streak_rank <=  3

"""
solution = pd.read_sql(query, conn)

In [100]:
# Compare the two results
are_equal = result_df.equals(solution)

# Print result based on the comparison
if are_equal:
    print("Correct!")
else:
    print("Try again!")

Try again!


### Problem Explanation
### Step 1: Remove deiplicate visits and filter visit date <= `2022-08-10`
```sql
SELECT
  user_id,
  date_visited
FROM user_streaks
WHERE date_visted <= '2022-08-10'
GROUP BY user_id, date_visited
```
Remove duplicate visits. A user might visit the site multiple times on the same date. We only care about distinct (user_id, date_visited) pairs.

- Filters visits up to 2022-08-10
- Groups by user_id and date_visited to deduplicate
- Equivalent to getting a distinct visit day per user

### Step 2: Detect whether a visit is part of a streak
```sql
SELECT
  user_id,
  date_visited,
  CASE
    WHEN JULIANDAY(date_visted) - JULIANDAY(LAG(date_visited) OVER(PARTITION BY user_id ORDER date_visited)) = 1 THEN 0
    ELSE 1
  END AS streak_marker
```
- For each user, sort their visits by date `PARTITION BY user_id ORDER date_visited`.
- Find the number of days difference between current visit and previous visit `date_visted - LAG(date_visited)`.
- If the difference in days = 1, then part of same streak (`streak_marker = 0`).
- Else, start of a new streak (`streak_marker = 1`).
- `JULIANDAY()` turns date into a numeric day count — so we can subtract.

### Step 3: Create streak ID
```sql
SELECT
  user_id,
  date_visited,
  streak_marker,
  SUM(streak_marker) OVER(PARTITION BY user_id ORDER BY date_visited) AS streak_id
FROM detected_streaks
```
Create a streak ID to group consecutive vistis.
- The cumulative SUM(streak_marker) increases only when a new streak starts.
- This creates a unique streak_id per contiguous date sequence.


### Step 4: Calculate streak length and rank users by streak length
```sql
SELECT
  user_id,
  streak_id,
  COUNT(*) AS streak_length,
  DENSE_RANK() OVER(ORDER BY COUNT(*) DESC) AS streak_rank
FROM streak_groups
GROUP BY user_id, streak_id
```
Compute the length of each streak and assign a rank.

- Groups by (`user_id`, `streak_id`) to count how many days each streak lasted.
- Uses `DENSE_RANK()` to assign streaks a rank based on their length (longer streak = higher rank).

### Step 5: Final output
```sql
SELECT
  user_id,
  streak_length
FROM streak_length
WHERE streak_rnk <= 3
```
Select all users and their streak lengths where the streak was among the top 3 longest streaks overall (with ties included via DENSE_RANK()).